In [1]:
from threading import Thread
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)

from transformers import DataCollatorWithPadding
from typing import List, Dict, Any
from transformers import PreTrainedTokenizerBase

c:\projects\polycrest-llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "Hankbeasley/Polycrest-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
from datasets import load_dataset
ds = load_dataset("Hankbeasley/polycoder")

In [3]:
input = ds['train'][0]['accept'].split('<think>')[0]
print(input)




<SystemPrompt>
  <Overview>
    You are a code-generation assistant designed to help users create high-quality, efficient, and well-documented code. You have access to various tools to aid in code analysis, debugging, and generation.
  </Overview>
  <Instructions>
    <Instruction>Provide accurate and efficient code solutions tailored to the user's problem.</Instruction>
    <Instruction>Follow best coding practices and include clear, helpful comments within the code.</Instruction>
    <Instruction>Explain code logic and structure clearly when requested.</Instruction>
    <Instruction>Leverage available tools to validate, analyze, and optimize code when applicable.</Instruction>
  </Instructions>
  <Capabilities>
    <Capability>Generate code in multiple programming languages</Capability>
    <Capability>Offer detailed explanations and documentation for code snippets</Capability>
    <Capability>Assist with debugging and code optimization</Capability>
    <Capability>Utilize integrated

In [4]:

input2 = "What can you do?<｜Assistant｜><think>"
tokens = tokenizer(input2, return_tensors="pt").to('cuda')
print(tokens)

decoded = tokenizer.decode(tokens['input_ids'][0])
print(decoded)

{'input_ids': tensor([[151646,   3838,    646,    498,    653,     30, 151645, 151648]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
<｜begin▁of▁sentence｜>What can you do?<｜Assistant｜><think>


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  
    torch_dtype=torch.bfloat16

).to("cuda")



In [6]:
modelO = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    device_map="auto",  
    torch_dtype=torch.bfloat16

).to("cuda")

In [ ]:
a=model.generate(tokens['input_ids'],  max_length=7000, pad_token_id=tokenizer.eos_token_id)
#print(a)
decoded = tokenizer.batch_decode(a, skip_special_tokens=True)
print(decoded[0])

What can you do?<｜Assistant｜><think>)




What’s

-

Do you

-

Think

-

Or

-

Do

you

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that

you

can

think

that


: 

In [ ]:


# Initialize the streamer
streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)



# Start a thread to generate predictions
generation_thread = Thread(target=model.generate, args=(tokens['input_ids'],), kwargs={'streamer': streamer})
generation_thread.start()

# Stream the output
for new_text in streamer:
    # Initialize the streamer
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)

    # Start a thread to generate predictions
    generation_thread = Thread(
        target=model.generate,
        args=(tokens['input_ids'],),
        kwargs={
            'attention_mask': tokens['attention_mask'],
            'streamer': streamer,
            'pad_token_id': tokenizer.eos_token_id
        }
    )
    generation_thread.start()

    # Stream the output
    for new_text in streamer:
        print(new_text, end='')

: 

: 